#Imports

In [ ]:
from enum import Enum
import random
from copy import deepcopy

Clases

In [ ]:
class Actions(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3
    NONE = -1

In [ ]:
class State:
    def __init__(self, values : list = None,indexOfZero : list= None):
        self._possibleValues = [0,1,2,3,4,5,6,7,8]
        self._goal = [[0,1,2],[3,4,5],[6,7,8]]
        if(indexOfZero == None):
            self.indexOfZero = [-1,-1]
        else:
            self.indexOfZero = indexOfZero
        if(values == None):
            self.values = [[-1,-1,-1],[-1,-1,-1],[-1,-1,-1]]
            self._randomValues()
        else:
            self.values = values
        self.solved = False
        self.exists = True
        self._setID()
        
    def _randomValues(self):
        for i in range(3):
            for j in range(3):
                value = random.choice(self._possibleValues)
                self.values[i][j] = value
                if(value == 0):
                    self.indexOfZero = [i,j]
                self._possibleValues.remove(value)
        self._possibleValues= []
    
    def toString(self) -> str:
        return '\n'.join(map(lambda x:' '.join(map(lambda y:str(y), x)), self.values))
    
    def isSolved(self) -> bool:
        for i in range(3):
            for j in range(3):
                if(self.values[i][j] != self._goal[i][j]):
                    return False
        return True
    
    # Move the zero to the direction specified
    def move(self,action:Actions):
        valid = True
        if(action == Actions.UP):
            valid = self._moveUp()
        elif(action == Actions.RIGHT):
            valid = self._moveRight()
        elif(action == Actions.DOWN):
            valid = self._moveDown()
        elif(action == Actions.LEFT):
            valid = self._moveLeft()
        self._setID()
        self.exists = valid
    
    def _moveUp(self) -> bool:
        if(self.indexOfZero[0] == 0):
            return False
        else:
            self.values[self.indexOfZero[0]][self.indexOfZero[1]] = self.values[self.indexOfZero[0]-1][self.indexOfZero[1]]
            self.values[self.indexOfZero[0]-1][self.indexOfZero[1]] = 0
            self.indexOfZero[0] -= 1
            return True
    
    def _moveRight(self) -> bool:
        if(self.indexOfZero[1] == 2):
            return False
        else:
            self.values[self.indexOfZero[0]][self.indexOfZero[1]] = self.values[self.indexOfZero[0]][self.indexOfZero[1]+1]
            self.values[self.indexOfZero[0]][self.indexOfZero[1]+1] = 0
            self.indexOfZero[1] += 1
            return True
    
    def _moveDown(self) -> bool:
        if(self.indexOfZero[0] == 2):
            return False
        else:
            self.values[self.indexOfZero[0]][self.indexOfZero[1]] = self.values[self.indexOfZero[0]+1][self.indexOfZero[1]]
            self.values[self.indexOfZero[0]+1][self.indexOfZero[1]] = 0
            self.indexOfZero[0] += 1
            return True
    
    def _moveLeft(self) -> bool:
        if(self.indexOfZero[1] == 0):
            return False
        else:
            self.values[self.indexOfZero[0]][self.indexOfZero[1]] = self.values[self.indexOfZero[0]][self.indexOfZero[1]-1]
            self.values[self.indexOfZero[0]][self.indexOfZero[1]-1] = 0
            self.indexOfZero[1] -= 1
            return True
        
    # Get a ID for the current state
    def _setID(self) -> int:
        id = int(''.join(map(lambda x:''.join(map(lambda y:str(y), x)), self.values)))
        self.id = id
        

In [ ]:
class Tree:
    # Create a tree of 4 states/children
    def __init__(self, state : State = None, children : list= None, parent : State= None,cost : int = 0,depth : int= 0,limit : int = 0):
        if(state == None):
            self.state = State()
        else:
            self.state = state
        if(children == None):
            self.children = [None,None,None,None]
        else:
            self.children = children
        self.parent = parent
        self.cost = cost
        self.depth = depth
        if(self.parent != None):
            self.limit = self.parent.limit
        else:
            self.limit = limit

    def toString(self):
        return self.state.toString()
    
    # Expand Solution
    def expand(self,depth : int = 1):
            if(depth == 0):
                return
            for i in range(4):
                if self.children[i] == None:
                    values = deepcopy(self.state.values) # copy of the values
                    indx = deepcopy(self.state.indexOfZero) # copy of the index of zero
                    newNode : State = State(values=values,indexOfZero=indx)
                    newNode.move(Actions(i))
                    if(newNode.exists):
                        
                        self.children[i] : Tree = Tree(newNode,parent=self,cost=self.cost+1,depth =self.depth+1)
                        if(depth-1):
                            self.children[i].expand(depth-1)
    
    # DFS Search
    def dfs(self, visited : list):
        # If we are not in the limit
        if(self.depth <= self.limit):
            # Adding the current state to the visited list
            visited.append(self.state.values)
            
            # Check if the state is the goal
            if(self.state.isSolved()):
                print("\t\t--Solucion encontrada--")
                print(f'Profunidad: {self.depth}')
                print(self.state.toString())
                print("".rjust(50,"-"))
                return self.state
            
            # Expand  and iterate throgh childs if not in limit
            if(self.depth != self.limit):
                self.expand(1)
                for i in range(4):
                    currentChild = self.children[i]
                    if currentChild == None:
                        continue
                    if currentChild.state.values in visited:
                        continue
                    result = self.children[i].dfs(visited)
                    if(result != None):
                        return result
            return None
        else:
            return None

Main Code

In [ ]:
# Initial State of the puzzle
s = State()
print("\t\t--Estado Inicial--")
s.values = [ [1,2,0], [3,4,5], [6,7,8] ]
s.indexOfZero = [0,2]
print(s.toString())
MAX_DEPTH = 50
t = Tree(state = s,limit=MAX_DEPTH)
print(f'Profundidad maxima: {t.limit}')
#Expand Tree Up to 12 Times
t.dfs(visited=[])
print("\t\t-- Ejecución Terminada --\n\n")